## Messy situation: Column headers are values, not variable names

***This is one of the more common messy data situations, so also one of the more common manipulations you need to get data into a tidy form!***

For example, many government data sets are in a format which is **good for visual lookup, but not for analysis and exploration**, with measurements in various years spread across multiple columns.

---

*To preserve the mystery, select from the notebook menus*

`Edit -> Clear All Outputs`

---

### A toy example – years as column headers

Let's define a simple, small DataFrame with that structure:

In [1]:
import pandas as pd

In [2]:
df = pd.DataFrame({'state':['Maine','Alaska','Ohio'],
                  '2009':[1,2,3],
                  '2010':[4,5,6],
                  '2011':[7,8,9]})
df

,state,2009,2010,2011
0,Maine,1,4,7
1,Alaska,2,5,8
2,Ohio,3,6,9


## Transforming from *wide* into *tall* format

The problems with this format are 

- **The column headers are really a Dimension that should be in its own *Year* column**
- **The values, spread across the multiple rows and columns in the body of the table, are a Measure that should be in a single column.**

Confusingly, every language seems to have its own term for this process / transformation:

- **In Pandas you do a "melt"**
- In the R `tidyr` package this is a "gather"
- In OpenRefine it's a "Transpose->Transpose cells across columns into rows..." operation
- In Tableau this is called a "Pivot"
- Many call this process "un-pivoting", since a *Pivot Table* in Excel (and Pandas, actually) converts data in the opposite direction, from the *tall* format into *wide*. 


### To `.melt()` you need to minimally supply the id_vars

The argument name for the list of columns that don't get melted is `id_vars`. This argument is the default, so you can choose to leave out the `id_vars=`, but it's not a bad idea in your code to spell this out explicitly. 

- The `id_vars` "identify" the rest of the columns in each original row. **They are the IDs, or the key fields, which uniquely identify the rows in the original table.**
- All the rest of the columns will get melted / un-pivoted

*Notice that*

- *the column headers by default end up in a column called "variable"*
- *the table body values end up in a column called "value"*
- *the id_vars get repeated for each original column*

In [3]:
df2 = df.melt(id_vars=['state'])
df2

,state,variable,value
0,Maine,2009,1
1,Alaska,2009,2
2,Ohio,2009,3
3,Maine,2010,4
4,Alaska,2010,5
5,Ohio,2010,6
6,Maine,2011,7
7,Alaska,2011,8
8,Ohio,2011,9


### Or just specify the columns to be melted

**If you don't want to keep any ID columns**, and just melt a few, you can just specify the value columns to melt.

In [4]:
df.melt(value_vars=['2009','2010','2011'])

,variable,value
0,2009,1
1,2009,2
2,2009,3
3,2010,4
4,2010,5
5,2010,6
6,2011,7
7,2011,8
8,2011,9


## More complete `.melt()` statement

You can explicitly specify any combination of these:

- list of columns that don't get melted (and, thus, will get repeated): `id_vars=`
- list of columns that get melted from columns into rows: `value_vars=`
- name you want for the column that used to be column headers: `var_name=`
- name you want for the column that used to be the table body values: `value_name=`


In [5]:
df2 = df.melt(id_vars=['state'], 
              value_vars=['2009','2010','2011'], 
              var_name='year', 
              value_name='number')
df2

,state,year,number
0,Maine,2009,1
1,Alaska,2009,2
2,Ohio,2009,3
3,Maine,2010,4
4,Alaska,2010,5
5,Ohio,2010,6
6,Maine,2011,7
7,Alaska,2011,8
8,Ohio,2011,9


---

## Sorting a DataFrame

You can sort the rows of a DataFrame according to any column, or ordered combination of columns.

- Like most DataFrame methods (functions), this returns a DataFrame, it doesn't change anthing in place.
- So, **the results won't be saved unless you assign the results of the sort to a variable or run it with the argument `inplace=True`!**
- Sometimes this is useful to just inspect a DataFrame in a certain sort order, when you don't really need to save the results.
- Notice that the default is to sort in "ascending" order, or `ascending=True`. 
- To get descending, you need to specify `ascending=False`.

*Notice that the Index doesn't get regenerated in the new sorted order by default. This is so you can do something like add another column from a Series and things won't be messed up just because you sorted the main DataFrame first!*


In [6]:
df2.sort_values(['state'])

,state,year,number
1,Alaska,2009,2
4,Alaska,2010,5
7,Alaska,2011,8
0,Maine,2009,1
3,Maine,2010,4
6,Maine,2011,7
2,Ohio,2009,3
5,Ohio,2010,6
8,Ohio,2011,9


#### See, it displayed `df2` in the sorted order above, but the sort didn't really get saved!

In [7]:
df2

,state,year,number
0,Maine,2009,1
1,Alaska,2009,2
2,Ohio,2009,3
3,Maine,2010,4
4,Alaska,2010,5
5,Ohio,2010,6
6,Maine,2011,7
7,Alaska,2011,8
8,Ohio,2011,9


### Sorting rows by multiple columns

Here we'll first sort the rows "descending" by 'year', and then within each year sort by 'state'. It's not necessary, but it's slightly more readable if you specify the argument name `by`.

In [8]:
df2 = df2.sort_values(by=['year','state'], ascending=False)
df2

,state,year,number
8,Ohio,2011,9
6,Maine,2011,7
7,Alaska,2011,8
5,Ohio,2010,6
3,Maine,2010,4
4,Alaska,2010,5
2,Ohio,2009,3
0,Maine,2009,1
1,Alaska,2009,2


#### `ascending` can also take a list the same length as `by`

In [9]:
df2 = df2.sort_values(by=['year','state'], ascending=[False,True])
df2

,state,year,number
7,Alaska,2011,8
6,Maine,2011,7
8,Ohio,2011,9
4,Alaska,2010,5
3,Maine,2010,4
5,Ohio,2010,6
1,Alaska,2009,2
0,Maine,2009,1
2,Ohio,2009,3


---

## Chaining commands

It's more efficient to chain commands together instead of creating intermediate variables when you don't really need them. *(By the way, it also helps you avoid the annoying and confusing `SettingWithCopyWarning` referenced in [DataFrameAccess](DataFrameAccess.ipynb).)*

I first learned about this in the article [4 Pandas Anti-Patterns](https://www.aidancooper.co.uk/pandas-anti-patterns/) by Aidan Cooper. He references and credits Matt Harrison's excellent book [Effective Pandas](https://store.metasnake.com/effective-pandas-book?xgtab&). *(Chapter 35 covers how to debug chained commands.)*

### Melt plus sort all in one command

One of the keys to chaining in a readable way is that the Python interpreter doesn't care if you break lines or have consistent indentation for things in parentheses.

In [10]:
df_tidy_sorted = (df
 .melt(
     id_vars=['state'], 
     value_vars=['2009','2010','2011'], 
     var_name='year', 
     value_name='number')
 .sort_values(
     by=['year','state'], 
     ascending=[False,True])
)

df_tidy_sorted

,state,year,number
7,Alaska,2011,8
6,Maine,2011,7
8,Ohio,2011,9
4,Alaska,2010,5
3,Maine,2010,4
5,Ohio,2010,6
1,Alaska,2009,2
0,Maine,2009,1
2,Ohio,2009,3
